<a href="https://colab.research.google.com/github/hasmeu/bounce/blob/main/Lyrics_from_Genre_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.8 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from datasets import load_dataset

In [3]:
dataset = load_dataset("D3STRON/music_lyrics_500k")

README.md:   0%|          | 0.00/500 [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/176M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/176M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/35.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/500000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
lyrics = [item["Lyric"] for item in dataset["train"]]
genres = [item["genre"] for item in dataset["train"]]

import pandas as pd
#sorts lyrics and genres
df = pd.DataFrame({'lyrics': lyrics, 'genre': genres})
df_sorted = df.sort_values(by=['genre'])
df_sorted = df_sorted.reset_index(drop=True)


In [16]:
df_sorted.iloc[]

'I hope you hear my cry Because Ive been calling you before I just want your voice on phone I think where can you be If you know what I am feeling Girl give me just one more chance O baby I need to be the one... Who has those lips to kiss? A cold invades my soul And I wonder youre so far Darling come and heat my heart For many thousand roads I will ride to be with you Just to be the best for you O baby tell me I aint wrong Who has your body to touch now? Open your eyes to me Open your heart to love Give a chance for we both Open your mind to me Open your hands to hope Give a chance for our love'

This is the data preparation.

In [17]:
!pip install transformers -q

In [19]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")  # Or any other suitable tokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [21]:
 # Tokenize the 'lyrics' column in your DataFrame
df_sorted['tokens'] = df_sorted['lyrics'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=512))

KeyboardInterrupt: 

In [ ]:
#creates input output pairs
input_sequences = df_sorted['genre'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, truncating=True, max_length=512)).tolist()
output_sequences = df_sorted['tokens'].tolist()

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
#padding/truncation
max_length = max(len(seq) for seq in input_sequences + output_sequences)  # Adjust if needed

input_sequences = pad_sequences(input_sequences, maxlen=max_length, padding='post', truncating='post')
output_sequences = pad_sequences(output_sequences, maxlen=max_length, padding='post', truncating='post')

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(input_sequences, output_sequences, test_size=0.2, random_state=42)  # Adjust test_size as needed

# You can further split X_train, y_train into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)  # Adjust validation_size as needed